In [27]:
import nltk
nltk.download('punkt',quiet=True)
nltk.download("stopwords",quiet=True)
import string 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import operator


def extract_text(s,find_tags = True, remove_separators = True):
    new_s = s[::]
    if(find_tags):
        start = [0,0] #Start is inclusive 
        end = [len(new_s)+1,0] #End is exclusive
        for i in range(len(s)):
            if(i + 7 <= len(s) and s[i:i+7] == "<TITLE>"):
                start[0] = i+7
            elif(i + 8 <= len(s) and s[i:i+8] == "</TITLE>"):
                end[0] = i
            elif(i + 6 <= len(s) and s[i:i+6] == "<TEXT>"):
                start[1] = i+6
            elif(i + 7 <= len(s) and s[i:i+7] == "</TEXT>"):
                end[1] = i
        new_s = s[start[0]:end[0]] + " " + s[start[1]:end[1]]
    if(remove_separators):
        new_s = " ".join(new_s.split("\n"))
        new_s = " ".join(new_s.split("-"))
    return new_s

def pre_process_text(new_s, remove_duplicates = False):
    new_s = new_s.lower()
    translate_table = dict((ord(char), " ") for char in string.punctuation)   
    new_s = new_s.translate(translate_table)
    li = word_tokenize(new_s)
    stop_words = set(stopwords.words("english"))
    filter_li = []
    for words in li:
        if(words not in stop_words):
            filter_li.append(words)
    if(remove_duplicates):
        filter_li = list(set(filter_li))
    return filter_li

def getNum(i):
    return (4-len(str(i)))*"0" + str(i)

def get_vocabulary(dataset):
    for i in dataset:
        for j in i.split():
            if(j not in vocab):
                vocab.append(j)

#------------------------------------------#### MAIN ####------------------------------------------#
overwrite_file = True
sample_number = 0
vocab = []
doc_list = []
id = 0

for j in range(1,1401): # 1,1401
    file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(j),"r")
    file_content = file.read()
    file.close()
    extracted_content = extract_text(file_content,find_tags=True,remove_separators=True)
    if(sample_number < 5):
        sample_number+=1
        print("Original File Content:")
        print(file_content)
        print("***************")
        print("Raw Extracted Data:")
        print(extracted_content)
        print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    if(overwrite_file):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(j),"w")
        file.write(extracted_content)
        file.close()


total_documents = 0
for i in range(1,1401):
    total_documents = total_documents+1
    file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
    file_content = file.read()
    file.close()
    extracted_content = pre_process_text(file_content, remove_duplicates=False)
    get_vocabulary(extracted_content)
    if(sample_number < 5):
        sample_number+=1
        print("Before Preprocessing:")
        print(file_content)
        print("***************")
        print("After Preprocessing: (Comma Separated Tokens)")
        for k in extracted_content:
            print(k,end=", ")
        print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")




        




Original File Content:
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects

In [34]:
#TFIDF MATRIX
import time

def getNum(i):
    return (4-len(str(i)))*"0" + str(i)

def get_vocabulary(dataset):
    for i in dataset:
        for j in i.split():
            if(j not in vocab):
                vocab.append(j)

def get_matrix():
    matrix = {}
    for i in range(1,1401):
        newlist = {}
        for k in range(len(vocab)):
            newlist[vocab[k]] = 0
        matrix[getNum(i)] = newlist.copy()
    matrix["num_docs"] = newlist.copy()
    return matrix


def binary_tfidf():
    start = time.time()
    for i in range(1,1401):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
        file_content = file.read()
        file.close()
        extracted_content = pre_process_text(file_content, remove_duplicates=False)
        for j in extracted_content:
            matrix_binary[getNum(i)][j] = 1

    for word in vocab:
        shu = 0
        lst = []
        for i in range(1,1401):
            if (matrix_binary[getNum(i)][word] > 0):
                shu = shu+1
                if getNum(i) not in lst:
                    lst.append(getNum(i))
        postinglist_binary[word] = lst    
        matrix_binary["num_docs"][word] = shu

    for i in matrix_binary:
        for j in matrix_binary[i]:
            tf_value = matrix_binary[i][j]
            idf_value = np.log10(total_documents/(matrix_binary["num_docs"][j]+1))
            tf_idf_value = tf_value*idf_value
            matrix_binary[i][j] = tf_idf_value

    end = time.time()
    print("Binary Scheme time:",end-start)


def rawcount_tfidf():
    start = time.time()
    for i in range(1,1401):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
        file_content = file.read()
        file.close()
        extracted_content = pre_process_text(file_content, remove_duplicates=False)
        for j in extracted_content:
            matrix_raw[getNum(i)][j] += 1

    for word in vocab:
        shu = 0
        lst = []
        for i in range(1,1401):
            if (matrix_raw[getNum(i)][word] > 0):
                shu = shu+1
                if getNum(i) not in lst:
                    lst.append(getNum(i))
        postinglist_raw[word] = lst  
        matrix_raw["num_docs"][word] = shu

    for i in matrix_raw:
        for j in matrix_raw[i]:
            tf_value = matrix_raw[i][j]
            idf_value = np.log10(total_documents/(matrix_raw["num_docs"][j]+1))
            tf_idf_value = tf_value*idf_value
            matrix_raw[i][j] = tf_idf_value
    end = time.time()
    print("Raw Count Scheme time:",end-start)


def termfreq_tfidf():
    start = time.time()
    for i in range(1,1401):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
        file_content = file.read()
        file.close()
        extracted_content = pre_process_text(file_content, remove_duplicates=False)
        total_terms = 0
        for j in extracted_content:
            total_terms += 1
            matrix_termfreq[getNum(i)][j] += 1
        for j in matrix_termfreq[getNum(i)]:
            ftd = matrix_termfreq[getNum(i)][j]
            matrix_termfreq[getNum(i)][j] = (ftd)/(total_terms-ftd+1)

    for word in vocab:
        shu = 0
        lst = []
        for i in range(1,1401):
            if (matrix_termfreq[getNum(i)][word] > 0):
                shu = shu+1
                lst.append(getNum(i))
        postinglist_termfreq[word] = lst  
        matrix_termfreq["num_docs"][word] = shu

    for i in matrix_termfreq:
        for j in matrix_termfreq[i]:
            tf_value = matrix_termfreq[i][j]
            idf_value = np.log10(total_documents/(matrix_termfreq["num_docs"][j]+1))
            tf_idf_value = tf_value*idf_value
            matrix_termfreq[i][j] = tf_idf_value

    end = time.time()
    print("Term Frequency Scheme time:",end-start)       

def lognorm_tfidf():
    start = time.time()
    for i in range(1,1401):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
        file_content = file.read()
        file.close()
        extracted_content = pre_process_text(file_content, remove_duplicates=False)
        for j in extracted_content:
            matrix_lognorm[getNum(i)][j] += 1

    for word in vocab:
        shu = 0
        lst = []
        for i in range(1,1401):
            if (matrix_lognorm[getNum(i)][word] > 0):
                shu = shu+1
                lst.append(getNum(i))
        postinglist_lognorm[word] = lst 
        matrix_lognorm["num_docs"][word] = shu

    for i in matrix_lognorm:
        if(i!="num_docs"):
            for j in matrix_lognorm[i]:
                ftd = matrix_lognorm[i][j]
                matrix_lognorm[i][j] = np.log10(1+ftd)


    for i in matrix_lognorm:
        for j in matrix_lognorm[i]:
            tf_value = matrix_lognorm[i][j]
            idf_value = np.log10(total_documents/(matrix_lognorm["num_docs"][j]+1))
            tf_idf_value = tf_value*idf_value
            matrix_lognorm[i][j] = tf_idf_value

    end = time.time()
    print("Log Normalization Scheme time:",end-start)


def double_lognorm_tfidf():
    start = time.time()
    for i in range(1,1401):
        file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
        file_content = file.read()
        file.close()
        extracted_content = pre_process_text(file_content, remove_duplicates=False)
        for j in extracted_content:
            matrix_doublenorm[getNum(i)][j] += 1
    for word in vocab:
        shu = 0
        lst = []
        for i in range(1,1401):
            if (matrix_doublenorm[getNum(i)][word] > 0):
                shu = shu+1
                lst.append(getNum(i))
        postinglist_doublenorm[word] = lst 
        matrix_doublenorm["num_docs"][word] = shu
    for i in matrix_doublenorm:
        if(i!="num_docs"):
            max_vals=matrix_doublenorm[i].values()
            l=sorted(max_vals)
            max1 = l[-1]
            max2 = l[-2]
            if(max2 == 0):
                max2 = max1
            for j in matrix_doublenorm[i]:
                ftd = matrix_doublenorm[i][j]
                if(ftd == max1):
                    matrix_doublenorm[i][j] = 0.5+0.5*(ftd/max2)
                else:
                    matrix_doublenorm[i][j] = 0.5+0.5*(ftd/max1)
    for i in matrix_doublenorm:
        for j in matrix_doublenorm[i]:
            tf_value = matrix_doublenorm[i][j]
            idf_value = np.log10(total_documents/(matrix_doublenorm["num_docs"][j]+1))
            tf_idf_value = tf_value*idf_value
            matrix_doublenorm[i][j] = tf_idf_value

    end = time.time()
    print("Double Normalization Scheme time:",end-start)


def tfidf_score_binary(query):
    query_dict_tf = {}
    newlist = {}
    for k in range(len(vocab)):
        newlist[vocab[k]] = 0
    query_dict_tf = newlist.copy()

    for i in query_dict_tf:
        if i in query:
            query_dict_tf[i] = 1
    for i in query_dict_tf:
        tf_val = query_dict_tf[i]
        idf_val = np.log10(total_documents/(len(postinglist_binary[i])+1))
        tfidf_val = tf_val*idf_val
        query_dict_tf[i] = tfidf_val
    query_tf = []
    for i in query_dict_tf:
       query_tf.append(query_dict_tf[i])  
    query_tfidf_vector = np.array(query_tf)
    tfidf_score_binary = {} 
    for i in matrix_binary:
        if(i != "num_docs"):
            doc = []
            for j in matrix_binary[i]:
                doc.append(matrix_binary[i][j])
            doc_tfidf_vector =  np.array(doc)
            tfidf_score = np.dot(query_tfidf_vector,doc_tfidf_vector)
        tfidf_score_binary[i] = tfidf_score    
    sorted_tfidf_score_binary = dict( sorted(tfidf_score_binary.items(), key=operator.itemgetter(1),reverse=True))
    ctr=0
    print("----------------------------------------SCHEME:Binary-------------------------------------------")
    for i in sorted_tfidf_score_binary:
        print("Document: ", i , ", Tfidf score: ",sorted_tfidf_score_binary[i])  
        ctr += 1
        if ctr == 5:
            break      

def tfidf_score_raw(query):
    query_dict_tf = {}
    newlist = {}
    for k in range(len(vocab)):
        newlist[vocab[k]] = 0
    query_dict_tf = newlist.copy()

    for i in query_dict_tf:
        if i in query:
            query_dict_tf[i] += 1
    for i in query_dict_tf:
        tf_val = query_dict_tf[i]
        idf_val = np.log10(total_documents/(len(postinglist_raw[i])+1))
        tfidf_val = tf_val*idf_val
        query_dict_tf[i] = tfidf_val
    query_tf = []
    for i in query_dict_tf:
       query_tf.append(query_dict_tf[i])  
    query_tfidf_vector = np.array(query_tf)
    tfidf_score_raw = {}    
    for i in matrix_raw:
        if(i != "num_docs"):
            doc = []
            for j in matrix_raw[i]:
                doc.append(matrix_raw[i][j])
            doc_tfidf_vector =  np.array(doc)
            tfidf_score = np.dot(query_tfidf_vector,doc_tfidf_vector)
        tfidf_score_raw[i] = tfidf_score    
    sorted_tfidf_score_raw = dict( sorted(tfidf_score_raw.items(), key=operator.itemgetter(1),reverse=True))
    ctr=0
    print()
    print("----------------------------------------SCHEME:Raw Count-------------------------------------------")
    for i in sorted_tfidf_score_raw:
        print("Document: ", i , ", Tfidf score: ",sorted_tfidf_score_raw[i])  
        ctr += 1
        if ctr == 5:
            break 

def tfidf_score_termfreq(query):
    query_dict_tf = {}
    newlist = {}
    for k in range(len(vocab)):
        newlist[vocab[k]] = 0
    query_dict_tf = newlist.copy()
    total_terms_query = 0
    for i in query_dict_tf:
        if i in query:
            total_terms_query += 1
            query_dict_tf[i] += 1
    for i in query_dict_tf:
        ftdq = query_dict_tf[i]
        query_dict_tf[i] = ftdq/(total_terms_query-ftdq)
    for i in query_dict_tf:
        tf_val = query_dict_tf[i]
        idf_val = np.log10(total_documents/(len(postinglist_termfreq[i])+1))
        tfidf_val = tf_val*idf_val
        query_dict_tf[i] = tfidf_val
    query_tf = []
    for i in query_dict_tf:
       query_tf.append(query_dict_tf[i])  
    query_tfidf_vector = np.array(query_tf)
    tfidf_score_termfreq = {}    
    for i in matrix_termfreq:
        if(i != "num_docs"):
            doc = []
            for j in matrix_termfreq[i]:
                doc.append(matrix_termfreq[i][j])
            doc_tfidf_vector =  np.array(doc)
            tfidf_score = np.dot(query_tfidf_vector,doc_tfidf_vector)
        tfidf_score_termfreq[i] = tfidf_score    
    sorted_tfidf_score_termfreq = dict( sorted(tfidf_score_termfreq.items(), key=operator.itemgetter(1),reverse=True))
    ctr=0
    print()
    print("----------------------------------------SCHEME:Term Frequency-------------------------------------------")
    for i in sorted_tfidf_score_termfreq:
        print("Document: ", i , ", Tfidf score: ",sorted_tfidf_score_termfreq[i])  
        ctr += 1
        if ctr == 5:
            break 

def tfidf_score_lognorm(query):
    query_dict_tf = {}
    newlist = {}
    for k in range(len(vocab)):
        newlist[vocab[k]] = 0
    query_dict_tf = newlist.copy()
    for i in query_dict_tf:
        if i in query:
            query_dict_tf[i] += 1
    for i in query_dict_tf:
        ftdq = query_dict_tf[i]
        query_dict_tf[i] = np.log10(ftdq+1)
    for i in query_dict_tf:
        tf_val = query_dict_tf[i]
        idf_val = np.log10(total_documents/(len(postinglist_lognorm[i])+1))
        tfidf_val = tf_val*idf_val
        query_dict_tf[i] = tfidf_val
    query_tf = []
    for i in query_dict_tf:
       query_tf.append(query_dict_tf[i])  
    query_tfidf_vector = np.array(query_tf)
    tfidf_score_lognorm = {}    
    for i in matrix_lognorm :
        if(i != "num_docs"):
            doc = []
            for j in matrix_lognorm[i]:
                doc.append(matrix_lognorm[i][j])
            doc_tfidf_vector =  np.array(doc)
            tfidf_score = np.dot(query_tfidf_vector,doc_tfidf_vector)
        tfidf_score_lognorm [i] = tfidf_score    
    sorted_tfidf_score_lognorm  = dict( sorted(tfidf_score_lognorm .items(), key=operator.itemgetter(1),reverse=True))
    ctr=0
    print()
    print("----------------------------------------SCHEME:Log Nomralization-------------------------------------------")
    for i in sorted_tfidf_score_lognorm :
        print("Document: ", i , ", Tfidf score: ",sorted_tfidf_score_lognorm [i])  
        ctr += 1
        if ctr == 5:
            break 


def tfidf_score_doublenorm(query):
    query_dict_tf = {}
    newlist = {}
    for k in range(len(vocab)):
        newlist[vocab[k]] = 0
    query_dict_tf = newlist.copy()
    for i in query_dict_tf:
        if i in query:
            query_dict_tf[i] += 1
    max_vals=query_dict_tf.values()
    l=sorted(max_vals)
    max1 = l[-1]
    max2 = l[-2]
    if(max2 == 0):
        max2 = max1
    for i in query_dict_tf:
        ftdq = query_dict_tf[i]
        if(ftdq == max1):
            query_dict_tf[i] = 0.5+0.5*(ftdq/max2)
        else:
            query_dict_tf[i] = 0.5+0.5*(ftdq/max1)
    for i in query_dict_tf:
        tf_val = query_dict_tf[i]
        idf_val =  np.log10(total_documents/(len(postinglist_doublenorm[i])+1))
        tfidf_val = tf_val*idf_val
        query_dict_tf[i] = tfidf_val

    query_tf = []
    for i in query_dict_tf:
       query_tf.append(query_dict_tf[i])  
    query_tfidf_vector = np.array(query_tf)
    tfidf_score_doublenorm = {}    
    for i in matrix_doublenorm:
        if(i != "num_docs"):
            doc = []
            for j in matrix_doublenorm[i]:
                doc.append(matrix_doublenorm[i][j])
            doc_tfidf_vector =  np.array(doc)
            tfidf_score = np.dot(query_tfidf_vector,doc_tfidf_vector)
        tfidf_score_doublenorm[i] = tfidf_score    
    sorted_tfidf_score_doublenorm = dict( sorted(tfidf_score_doublenorm.items(), key=operator.itemgetter(1),reverse=True))
    ctr=0
    print()
    print("----------------------------------------SCHEME:Double Nomralization-------------------------------------------")
    for i in sorted_tfidf_score_doublenorm:
        print("Document: ", i , ", Tfidf score: ",sorted_tfidf_score_doublenorm[i])  
        ctr += 1
        if ctr == 5:
            break 
        
postinglist_binary = {}
postinglist_raw = {}
postinglist_termfreq = {}
postinglist_lognorm = {}
postinglist_doublenorm = {}
matrix_binary = {}
matrix_binary = get_matrix()  
binary_tfidf()
matrix_raw = {}
matrix_raw= get_matrix() 
rawcount_tfidf()
matrix_termfreq = {}
matrix_termfreq= get_matrix() 
termfreq_tfidf()
matrix_lognorm = {}
matrix_lognorm= get_matrix() 
lognorm_tfidf()
matrix_doublenorm = {}
matrix_doublenorm= get_matrix()
double_lognorm_tfidf()
query_tokens = "Calculate the experimental evaluation of theory of investigation to increase and make the maximum amount of approximations "
if query_tokens == "":
    print("Query is empty")
else:
    query_list = pre_process_text(query_tokens)
    tfidf_score_binary(query_list)
    tfidf_score_raw(query_list)
    tfidf_score_termfreq(query_list)
    tfidf_score_lognorm(query_list)
    tfidf_score_doublenorm(query_list)


Binary Scheme time: 16.81926131248474
Raw Count Scheme time: 16.734785556793213
Term Frequency Scheme time: 27.17898678779602
Log Normalization Scheme time: 26.030656576156616
Double Normalization Scheme time: 19.114404916763306
----------------------------------------SCHEME:Binary-------------------------------------------
Document:  0677 , Tfidf score:  6.698194669434006
Document:  0946 , Tfidf score:  6.300055546427604
Document:  1366 , Tfidf score:  6.240245450316783
Document:  0815 , Tfidf score:  5.887458271855376
Document:  0001 , Tfidf score:  5.708712778454509

----------------------------------------SCHEME:Raw Count-------------------------------------------
Document:  1291 , Tfidf score:  15.517378538045824
Document:  0262 , Tfidf score:  13.705624037750864
Document:  1344 , Tfidf score:  11.7001008939
Document:  0156 , Tfidf score:  10.233549053655969
Document:  0673 , Tfidf score:  9.240419922840783

----------------------------------------SCHEME:Term Frequency------------

In [33]:
#JACCARD COEFFICIENT
query_tokens = "Calculate the experimental evaluation of theory of investigation to increase and make the maximum amount of approximations "
query_list = pre_process_text(query_tokens)
Jaccard_coeff = {}
for i in range(1,1401):
    file = open("CSE508_Winter2023_Dataset/cranfield"+getNum(i),"r")
    file_content = file.read()
    file.close()
    extracted_content = pre_process_text(file_content, remove_duplicates=False)
    query_list_set = set(query_list)
    extracted_content_set = set(extracted_content) 
    intersection_doc_query = query_list_set.intersection(extracted_content_set) 
    union_doc_query = query_list_set.union(extracted_content_set)
    jc = len(intersection_doc_query)/len(union_doc_query)
    Jaccard_coeff[getNum(i)] = jc
Jaccard_coeff_sort= dict(sorted(Jaccard_coeff.items(), key=operator.itemgetter(1),reverse=True))
ctr = 0
for i in Jaccard_coeff_sort:
    print("Document:",i,", Jaccard coeffecient:",Jaccard_coeff_sort[i])
    ctr +=1
    if(ctr == 10):
        break

Document: 1045 , Jaccard coeffecient: 0.09523809523809523
Document: 0339 , Jaccard coeffecient: 0.08823529411764706
Document: 0932 , Jaccard coeffecient: 0.08571428571428572
Document: 0176 , Jaccard coeffecient: 0.08163265306122448
Document: 0001 , Jaccard coeffecient: 0.07692307692307693
Document: 0137 , Jaccard coeffecient: 0.06896551724137931
Document: 0019 , Jaccard coeffecient: 0.06818181818181818
Document: 0250 , Jaccard coeffecient: 0.06666666666666667
Document: 0920 , Jaccard coeffecient: 0.06666666666666667
Document: 0251 , Jaccard coeffecient: 0.06521739130434782
